In [90]:
import sys


def print_line(*args):
    """ Inline print and go to the begining of line
    """
    args1 = [str(arg) for arg in args]
    str_ = ' '.join(args1)
    print('\r' + str_, end='')

In [91]:
import tensorflow as tf

# List all physical devices, including GPUs
physical_devices = tf.config.list_physical_devices()
print("Physical devices available:")
for device in physical_devices:
    print(device)


Physical devices available:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [92]:
from typing import List, Tuple, Union, Dict

import numpy as np

In [93]:
import os
import pickle


data_path = '/Users/naveenverma/Desktop/NewStart/Dataset/a3-data'

train_sentences = open(os.path.join(data_path, 'train.txt')).readlines()
valid_sentences = open(os.path.join(data_path, 'valid.txt')).readlines()
test_sentences = open(os.path.join(data_path, 'input.txt')).readlines()
print('number of train sentences:', len(train_sentences))
print('number of valid sentences:', len(valid_sentences))
print('number of test sentences:', len(test_sentences))

number of train sentences: 42068
number of valid sentences: 3370
number of test sentences: 3165


In [94]:
import re


class Preprocessor:
    def __init__(self, punctuation=True, url=True, number=True):
        self.punctuation = punctuation
        self.url = url
        self.number = number

    def apply(self, sentence: str) -> str:
        """ Apply the preprocessing rules to the sentence
        Args:
            sentence: raw sentence
        Returns:
            sentence: clean sentence
        """
        sentence = sentence.lower()
        sentence = sentence.replace('<unk>', '')
        if self.url:
            sentence = Preprocessor.remove_url(sentence)
        if self.punctuation:
            sentence = Preprocessor.remove_punctuation(sentence)
        if self.number:
            sentence = Preprocessor.remove_number(sentence)
        sentence = re.sub(r'\s+', ' ', sentence)
        return sentence

    @staticmethod
    def remove_punctuation(sentence: str) -> str:
        """ Remove punctuations in sentence with re
        Args:
            sentence: sentence with possible punctuations
        Returns:
            sentence: sentence without punctuations
        """
        sentence = re.sub(r'[^\w\s]', ' ', sentence)
        return sentence

    @staticmethod
    def remove_url(sentence: str) -> str:
        """ Remove urls in text with re
        Args:
            sentence: sentence with possible urls
        Returns:
            sentence: sentence without urls
        """
        sentence = re.sub(r'(https|http)?://(\w|\.|/|\?|=|&|%)*\b', ' ', sentence)
        return sentence

    @staticmethod
    def remove_number(sentence: str) -> str:
        """ Remove numbers in sentence with re
        Args:
            sentence: sentence with possible numbers
        Returns:
            sentence: sentence without numbers
        """
        sentence = re.sub(r'\d+', ' ', sentence)
        return sentence

In [95]:
class Tokenizer:
    def __init__(self, sos_token='<s>', eos_token='</s>', pad_token='<pad>', unk_token='<unk>', mask_token='<mask>'):
        # Special tokens.
        self.sos_token = sos_token
        self.eos_token = eos_token
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.mask_token = mask_token

        self.vocab = { sos_token: 0, eos_token: 1, pad_token: 2, unk_token: 3, mask_token: 4 }  # token -> id
        self.inverse_vocab = { 0: sos_token, 1: eos_token, 2: pad_token, 3: unk_token, 4: mask_token }  # id -> token
        self.token_occurrence = { sos_token: 0, eos_token: 0, pad_token: 0, unk_token: 0, mask_token: 0 }  # token -> occurrence

        self.preprocessor = Preprocessor()

    @property
    def sos_token_id(self):
        """ Create a property method.
            You can use self.sos_token_id or tokenizer.sos_token_id to get the id of the sos_token.
        """
        return self.vocab[self.sos_token]

    @property
    def eos_token_id(self):
        return self.vocab[self.eos_token]

    @property
    def pad_token_id(self):
        return self.vocab[self.pad_token]

    @property
    def unk_token_id(self):
        return self.vocab[self.unk_token]

    @property
    def mask_token_id(self):
        return self.vocab[self.mask_token]

    def __len__(self):
        """ A magic method that enable program to know the number of tokens by calling:
            ```python
            tokenizer = Tokenizer()
            num_tokens = len(tokenizer)
            ```
        """
        return len(self.vocab)

    def fit(self, sentences: List[str]):
        """ Fit the tokenizer using all sentences.
        1. Tokenize the sentence by splitting with spaces.
        2. Record the occurrence of all tokens
        3. Construct the token to index (self.vocab) map and the inversed map (self.inverse_vocab) based on the occurrence. The token with a higher occurrence has the smaller index

        Args:
            sentences: All sentences in the dataset.
        """
        n = len(sentences)
        for i, sentence in enumerate(sentences):
            if i % 100 == 0 or i == n - 1:
                print_line('Fitting Tokenizer:', (i + 1), '/', n)
            tokens = self.preprocessor.apply(sentence.strip()).split()
            if len(tokens) <= 1:
                continue
            for token in tokens:
                if token == '<unk>':
                    continue
                self.token_occurrence[token] = self.token_occurrence.get(token, 0) + 1
        print_line('\n')

        token_occurrence = sorted(self.token_occurrence.items(), key=lambda e: e[1], reverse=True)
        for token, occurrence in token_occurrence[:-5]:
            token_id = len(self.vocab)
            self.vocab[token] = token_id
            self.inverse_vocab[token_id] = token

        print('The number of distinct tokens:', len(self.vocab))

    def encode(self, sentences: List[str]) -> List[List[int]]:
        """ Encode the sentences into token ids
            Note: 1. if a token in a sentence does not exist in the fit encoder, we ignore it.
                  2. If the number of tokens in a sentence is less than two, we ignore this sentence.
                  3. Note that, for every sentence, we will add an sos_token, i.e., the id of <s> at the start of the sentence,
                     and add an eos_token, i.e., the id of </s> at the end of the sentence.
        Args:
            sentences: Raw sentences
        Returns:
            sent_token_ids: A list of id list
        """
        n = len(sentences)
        sent_token_ids = []
        for i, sentence in enumerate(sentences):
            if i % 100 == 0 or i == n - 1:
                print_line('Encoding with Tokenizer:', (i + 1), '/', n)
            token_ids = []
            tokens = self.preprocessor.apply(sentence.strip()).split()
            for token in tokens:
                if token == '<unk>':
                    continue
                if token in self.vocab:
                    token_ids.append(self.vocab[token])
            if len(token_ids) <= 1:
                continue
            token_ids = [self.sos_token_id] + token_ids + [self.eos_token_id]
            sent_token_ids.append(token_ids)
        print_line('\n')
        return sent_token_ids

In [96]:
tokenizer = Tokenizer()
tokenizer.fit(train_sentences[:2])
print()

token_occurrence = sorted(tokenizer.token_occurrence.items(), key=lambda e: e[1], reverse=True)
for token, occurrence in token_occurrence[:10]:
    print(token, ':', occurrence)
print()
sent_token_ids = tokenizer.encode(train_sentences[:2])
print()
for original_sentence, token_ids in zip(train_sentences[:2], sent_token_ids):
    sentence = [tokenizer.inverse_vocab[token] for token in token_ids]
    print(original_sentence, sentence, '\n')

Fitting Tokenizer: 2 / 2
The number of distinct tokens: 44

n : 2
aer : 1
banknote : 1
berlitz : 1
calloway : 1
centrust : 1
cluett : 1
fromstein : 1
gitano : 1
guterman : 1

Encoding with Tokenizer: 2 / 2

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 ['<s>', 'aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro', 'quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack', 'food', 'ssangyong', 'swapo', 'wachter', '</s>'] 

 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 ['<s>', 'pierre', 'n', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov', 'n', '</s>'] 



In [97]:
tokenizer = Tokenizer()
tokenizer.fit(train_sentences)
train_token_ids = tokenizer.encode(train_sentences)
valid_token_ids = tokenizer.encode(valid_sentences)
test_token_ids = tokenizer.encode(test_sentences)
print(train_token_ids)

Fitting Tokenizer: 42068 / 42068
The number of distinct tokens: 9614
Encoding with Tokenizer: 42068 / 42068
Encoding with Tokenizer: 3370 / 3370
Encoding with Tokenizer: 3165 / 3165


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [98]:
from collections import defaultdict

def get_unigram_count(train_token_ids: List[List[int]]) -> Dict:
    """ Calculate the occurrence of each token in the dataset.

    Args:
        train_token_ids: each element is a list of token ids
    Return:
        unigram_count: A map from token_id to occurrence
    """
    unigram_count = defaultdict(int)
    for sentence_token_ids in train_token_ids:
        for token_id in sentence_token_ids:
            unigram_count[token_id] += 1
    return dict(unigram_count)


In [99]:
def get_bigram_count(train_token_ids: List[List[int]]) -> Dict[int, Dict]:
    """ Calculate the occurrence of bigrams in the dataset.

    Args:
        train_token_ids: each element is a list of token ids
    Return:
        bigram_count: A map from token_id to next token occurrence. Key: token_id, value: Dict[token_id -> occurrence]
                      For example, {
                          5: { 10: 5, 20: 4 }
                      } means (5, 10) occurs 5 times, (5, 20) occurs 4 times.
    """
    bigram_count = defaultdict(lambda: defaultdict(int))
    for sentence_token_ids in train_token_ids:
        for i in range(len(sentence_token_ids) - 1):
            current_token_id = sentence_token_ids[i]
            next_token_id = sentence_token_ids[i + 1]
            bigram_count[current_token_id][next_token_id] += 1
    return dict(bigram_count)


In [100]:
class BiGram:
    def __init__(self, unigram_count, bigram_count):
        self.unigram_count = unigram_count
        self.bigram_count = bigram_count

    def calc_prob(self, w1: int, w2: int) -> float:
        """ Calculate the probability of p(w2 | w1) using the BiGram model.

        Args:
            w1, w2: current token and next token
        Note:
            if prob you calculated is 0, you should return 1e-5.
        """
        # Calculate the probability of w2 given w1
        if w1 in self.bigram_count and w2 in self.bigram_count[w1]:
            count_w1_w2 = self.bigram_count[w1][w2]
            count_w1 = self.unigram_count[w1]
            if count_w1 != 0:
                prob = count_w1_w2 / count_w1
            else:
                prob = 0
        else:
            prob = 0

        # If prob is 0, return a small value instead to avoid zero probability
        if prob == 0:
            prob = 1e-5

        return prob


In [101]:
from scipy.optimize import curve_fit

def power_law(x, a, b):
    """ Power law to fit the number of occurrence
    """
    return a * np.power(x, b)

class GoodTuring(BiGram):
    def __init__(self, unigram_count, bigram_count, threshold=100):
        super().__init__(unigram_count, bigram_count)
        self.threshold = threshold
        self.bigram_Nc = self.calc_Nc()
        self.bi_c_star, self.bi_N = self.smoothing(self.bigram_Nc)

    def calc_Nc(self) -> Dict[int, Union[float, int]]:
        bigram_Nc = {}
        for _, next_counts in self.bigram_count.items():
            for count in next_counts.values():
                if count in bigram_Nc:
                    bigram_Nc[count] += 1
                else:
                    bigram_Nc[count] = 1
        self.replace_large_c(bigram_Nc)
        return bigram_Nc

    def replace_large_c(self, Nc):
        x, y = zip(*sorted(Nc.items(), reverse=True))
        popt, pcov = curve_fit(power_law, x, y, bounds=([0, -np.inf], [np.inf, 0]))
        a, b = popt
        max_count = max(Nc.keys())
        for c in range(self.threshold + 1, max_count + 2):
            Nc[c] = power_law(c, a, b)

    def smoothing(self, Nc: Dict[int, Union[float, int]]) -> Tuple[Dict[int, float], float]:
        c_star = {}
        N = 0
        max_count = max(Nc.keys())
        for count, occurrence in Nc.items():
            c_star[count] = (count + 1) * Nc.get(count + 1, 0) / Nc.get(count, 1)
            N += count * occurrence
        return c_star, N

    def calc_prob(self, w1, w2):
        prob = 0
        if w1 in self.bigram_count and w2 in self.bigram_count[w1]:
            count_w1_w2 = self.bigram_count[w1][w2]
            if count_w1_w2 in self.bi_c_star:
                smoothed_count = self.bi_c_star[count_w1_w2]
                if w1 in self.unigram_count:
                    count_w1 = self.unigram_count[w1]
                    prob = smoothed_count / count_w1
                else:
                    prob = smoothed_count / self.bi_N
        else:
            if w1 in self.unigram_count:
                count_w1 = self.unigram_count[w1]
                prob = 1 / (count_w1 + 1)  # Add-one smoothing
            else:
                prob = 1 / len(self.unigram_count)  # Uniform distribution
        return prob


In [102]:
class KneserNey(BiGram):
    def __init__(self, unigram_count, bigram_count, d=0.75):
        super().__init__(unigram_count, bigram_count)
        self.d = d
        self.lambda_ = self.calc_lambda()
        self.p_continuation = self.calc_p_continuation()

    def calc_lambda(self):
        """ Calculate the λ(w) """
        lambda_ = {}
        for w1 in self.bigram_count:
            lambda_[w1] = self.d * len(self.bigram_count[w1]) / self.unigram_count[w1]
        return lambda_

    def calc_p_continuation(self):
        """ Calculate the p_continuation(w) """
        numerator = {}
        for w1, next_counts in self.bigram_count.items():
            for _ in next_counts:
                numerator[_] = numerator.get(_, 0) + 1
        denominator = len(numerator)
        p_continuation = {}
        for w, count in numerator.items():
            p_continuation[w] = count / denominator
        return p_continuation

    def calc_prob(self, w1, w2):
        """ Calculate the probability of p(w2 | w1) using the Kneser-Ney model. """
        c_w1_w2 = self.bigram_count[w1].get(w2, 0)
        prob = max(c_w1_w2 - self.d, 0) / self.unigram_count[w1] + self.lambda_[w1] * self.p_continuation[w2]
        return prob


In [103]:
import math

def perplexity(model, token_ids):
    """ Calculate the perplexity score.

    Args:
        model: the model you want to evaluate (BiGram, GoodTuring, or KneserNey)
        token_ids: a list of validation token_ids
    Return:
        perplexity: the perplexity of the model on texts
    """
    log_probs = 0
    n = len(token_ids)
    n_words = 0
    for i, tokens in enumerate(token_ids):
        if i % 100 == 0 or i == n - 1:
            print_line('Calculating perplexity:', (i + 1), '/', n)
        # Calculate log probability of each token pair
        for j in range(1, len(tokens)):
            log_prob = math.log(model.calc_prob(tokens[j - 1], tokens[j]) + 1e-10)  # Add small value to avoid log(0)
            log_probs += log_prob
            n_words += 1

    # Calculate perplexity
    perp = math.exp(-log_probs / n_words)
    print('\n')

    return perp


In [104]:
unigram_count = get_unigram_count(train_token_ids)
bigram_count = get_bigram_count(train_token_ids)

In [105]:
bigram = BiGram(unigram_count, bigram_count)

# Perplexity
bigram_perplexity = perplexity(bigram, valid_token_ids)
print(f'The perplexity of Bigram is: {bigram_perplexity:.4f}')


Calculating perplexity: 3352 / 3352

The perplexity of Bigram is: 325.8347


In [106]:
# Initialize the GoodTuring object
gt = GoodTuring(unigram_count, bigram_count, threshold=100)

gt_perplexity = perplexity(gt, valid_token_ids)

# Print the perplexity value
print(f'The perplexity of Good Turing is: {gt_perplexity:.4f}')


Calculating perplexity: 3352 / 3352

The perplexity of Good Turing is: 95.1412


In [107]:
kn = KneserNey(unigram_count, bigram_count, d=0.75)

# Perplexity
kn_perplexity = perplexity(kn, valid_token_ids)
print(f'The perplexity of Kneser-Ney is: {kn_perplexity:.4f}')

Calculating perplexity: 3352 / 3352

The perplexity of Kneser-Ney is: 62.5908


In [108]:
def predict(model: 'BiGram', w1: int, vocab_size: int):
    """ Predict the w2 with the highest probability given w1

    Args:
        model: A BiGram, GoodTuring, or KneserNey model that has the calc_prob function
        w1: current word
        vocab_size: the number of tokens in the vocabulary
    """
    result = None
    highest_prob = 0
    for w2 in range(1, vocab_size):
        prob = model.calc_prob(w1, w2)
        if prob > highest_prob:
            highest_prob = prob
            result = w2
    return result


In [109]:
np.random.seed(12345)

vocab_size = len(tokenizer)
indexes = np.random.choice(len(test_token_ids), 10, replace=False)
for i in indexes:
    token_ids = test_token_ids[i][1:-1]
    print(' '.join([tokenizer.inverse_vocab[token_id] for token_id in token_ids]) + ' ____')
    pred = predict(gt, token_ids[-1], vocab_size)
    print(f'predicted last token: {tokenizer.inverse_vocab[pred]}')
    print('---------------------------------------------')

sharply falling stock prices do reduce consumer wealth damage business ____
predicted last token: </s>
---------------------------------------------
but robert an official of the association said no ____
predicted last token: longer
---------------------------------------------
it also has interests in military electronics and marine ____
predicted last token: s
---------------------------------------------
first chicago since n has reduced its loans to such ____
predicted last token: as
---------------------------------------------
david m jones vice president at g ____
predicted last token: s
---------------------------------------------
the n stock specialist firms on the big board floor ____
predicted last token: traders
---------------------------------------------
at the same time the business was hurt by ____
predicted last token: the
---------------------------------------------
salomon will cover the warrants by buying sufficient shares or ____
predicted last token: n
--------